In [1]:
import json
import csv

In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
input_path = "/content/drive/MyDrive/ Task4_Expert_Chatbot /arxiv-metadata-oai-snapshot.json"
output_path = "/content/drive/MyDrive/ Task4_Expert_Chatbot /computer_science_papers.csv"

In [5]:
import os
print(os.path.exists("/content/drive/MyDrive/ Task4_Expert_Chatbot /arxiv-metadata-oai-snapshot.json"))


True


In [6]:
with open(input_path, 'r', encoding='utf-8') as json_file, \
     open(output_path, 'w', encoding='utf-8', newline='') as csv_file:

    fieldnames = ['id', 'title', 'abstract', 'categories']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    for line in json_file:
        record = json.loads(line)
        if 'cs.' in record.get('categories', ''):
            writer.writerow({
                'id': record.get('id', ''),
                'title': record.get('title', '').replace('\n', ' ').strip(),
                'abstract': record.get('abstract', '').replace('\n', ' ').strip(),
                'categories': record.get('categories', '')
            })

In [7]:
!pip install -q sentence-transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 147.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 114.6 MB/s eta 0:00:00


In [8]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle


In [9]:
file_path = "/content/drive/MyDrive/ Task4_Expert_Chatbot /computer_science_papers.csv"
df = pd.read_csv(file_path)
df.dropna(subset=['abstract'], inplace=True)
df.reset_index(drop=True, inplace=True)

/tmp/ipython-input-106468218.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [10]:
model = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
embeddings = model.encode(df['abstract'][:10000].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [12]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

In [16]:
import os

save_path = '/content/drive/MyDrive/Task4_Expert_Chatbot'
os.makedirs(save_path, exist_ok=True)


In [17]:
faiss.write_index(index, f'{save_path}/cs_index.faiss')
df.to_pickle(f'{save_path}/cs_data.pkl')


In [18]:
faiss.write_index(index, '/content/drive/MyDrive/Task4_Expert_Chatbot/cs_index.faiss')
df.to_pickle('/content/drive/MyDrive/Task4_Expert_Chatbot/cs_data.pkl')

In [23]:
pip install streamlit faiss-cpu sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [19]:
import faiss

In [20]:
def search(query, top_k=5):
    query_embedding = model.encode([query])
    D, I = index.search(query_embedding, top_k)
    results = df.iloc[I[0]]
    return results

In [21]:
query = "transformer based models for natural language processing"
results = search(query)

for i, row in results.iterrows():
    print(f"\nTitle: {row['title']}\nCategories: {row['categories']}\nAbstract: {row['abstract'][:400]}...")




Title: The Generation of Textual Entailment with NLML in an Intelligent   Dialogue system for Language Learning CSIEC
Categories: cs.CL cs.AI cs.CY
Abstract: This research report introduces the generation of textual entailment within the project CSIEC (Computer Simulation in Educational Communication), an interactive web-based human-computer dialogue system with natural language for English instruction. The generation of textual entailment (GTE) is critical to the further improvement of CSIEC project. Up to now we have found few literatures related wit...

Title: Valence extraction using EM selection and co-occurrence matrices
Categories: cs.CL
Abstract: This paper discusses two new procedures for extracting verb valences from raw texts, with an application to the Polish language. The first novel technique, the EM selection algorithm, performs unsupervised disambiguation of valence frame forests, obtained by applying a non-probabilistic deep grammar parser and some post-processing to 

In [30]:
# Save both dataframe and FAISS index to a pickle file
import pickle
import os

In [32]:
os.makedirs("C:/Task4_Expert_Chatbot", exist_ok=True)

with open("C:/Task4_Expert_Chatbot/cs_data.pkl", "wb") as f:
    pickle.dump((df[:20000], index), f)


In [34]:


print("Current Working Directory:", os.getcwd())


folder_path = "C:/Task4_Expert_Chatbot"
os.makedirs(folder_path, exist_ok=True)

with open(os.path.join(folder_path, "cs_data.pkl"), "wb") as f:
    pickle.dump((df[:20000], index), f)

print("Pickle file saved successfully ✅")

Current Working Directory: /content
Pickle file saved successfully ✅
